<a href="https://colab.research.google.com/github/bryanzheng2222/Job-Scraping/blob/main/Job%20scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import requests, csv, os
from random import uniform
from hashlib import md5
from time import sleep
from datetime import date, datetime

In [43]:
def ERROR(skk): print("\033[91m{}\033[00m" .format(skk),end='')
def RED(skk): print("\033[91m{}\033[00m" .format(skk))
def GREEN(skk): print("\033[92m{}\033[00m" .format(skk))
def YELLOW(skk): print("\033[93m{}\033[00m" .format(skk))
def LIGHTGRAY(skk): print("\033[97m{}\033[00m" .format(skk),end='')

In [42]:
def writeCsv(name, data):
	with open(name + '.csv', 'a', newline='', encoding='utf_8_sig') as csvfile:
		fieldnames = ['company', 'job_title', 'job_type', 'job_location', 'job_link', 'job_description', 'period_from', 'period_to', 'logo', 'md5']
		writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
		if os.stat(name + '.csv').st_size == 0:
			writer.writeheader()
		writer.writerow(data)

In [11]:
def randomSleep(statusCode, content, s, e):
	rn = round(uniform(s, e), 4)
	LIGHTGRAY('[Status:' + str(statusCode) + ', Size:' + str(round((len(content)/1024),2)) + 'k' + ']')
	GREEN(' sleep for ' + str(rn) +' secound')
	sleep(rn)

In [12]:
def getPage(url, base_headers):
	try:
		R = requests.request("GET", url, headers=base_headers)
		if R.ok:
			# GREEN('OK')
			return R.text
		else:
			raise ValueError(R.reason)
	except Exception as e:
		RED('GET ERROR: ' + str(e.args))
	finally:
		randomSleep(R.status_code, R.content, 3, 9)

In [40]:
def postPage(url, base_headers, payload):
	try:
		R = requests.request("POST", url, headers=base_headers, data=payload)
		if R.ok:
			# print('OK')
			return R.text
		else:
			raise ValueError(R.reason)
	except Exception as e:
		RED('POST ERROR' + str(e.args))
	finally:
		randomSleep(R.status_code, R.content, 3, 9)

In [13]:
def addJob(data):
    result = add(data, 'jobs', 'job_id')
    print(result)

In [14]:
def md5List(name): return query_by_md5(name)

In [15]:
def getMd5(data): return md5((bytes(data, 'utf-8'))).hexdigest()

In [16]:
def toDay(): return date.today().strftime("%b-%d-%Y")

In [17]:
def dumpData(filename, data, mode='w'):
	f = open(filename, mode)
	f.write(data)
	f.close()

In [18]:
def dumpError(data):
	RED(data['error'])
	dumpData('error.txt', str(data) + '\r\n', 'a')


In [19]:
def formatDate(date, format):
	return datetime.strptime(date, format).strftime("%b-%d-%Y")

In [20]:

def formatHTML(data):
	return "".join(line.strip() for line in str(data).split("\n"))

In [39]:
based_headers = {
    'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
    'connection': "keep-alive",
    'upgrade-insecure-requests':"1",
    'accept': "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    'accept-encoding': "gzip, deflate, br",
    'accept-language': "en-GB,en-US;q=0.9,en;q=0.8,zh-CN;q=0.7,zh;q=0.6",
    'cache-control': "no-cache, max-age=0, must-revalidate, no-store",
    }
logo = https://jobs.fphcare.com/skins/fph-2017/img/logo.jpg
career_url = https://jobs.fphcare.com/search

SyntaxError: ignored

In [21]:
import ast, sys, traceback

In [22]:
sys.path.append('../')

In [24]:
from bs4 import BeautifulSoup
from configparser import RawConfigParser

In [30]:
pip install common

In [31]:
from common import dumpError, getPage, addJob, getMd5, md5List, toDay, formatDate, formatHTML, GREEN, YELLOW

ImportError: ignored

In [35]:
class fisherpaykel:
  def __init__(self):
    config = RawConfigParser()
    config.read('webConfig/fisherpaykel.ini')
    self.logo = config['fisherpaykel']['logo']
    self.career_url = config['fisherpaykel']['career_url']
    self.base_headers = ast.literal_eval(config['fisherpaykel']['base_headers'])
    self.soup = BeautifulSoup(getPage(self.career_url, self.base_headers),features = 'html.parser')


In [37]:
def findjob(self, name):
  jobList = self.soup.find ('div',class_= 'job-list').find_all('div',class_='clicker-wrapper')
  existJobList = md5List(name)

  for job in jobList:
    D = {}
    jobLink = job.h3.a.attrs['href']
    jobTitle = job.h3.a.string.strip()
    jobMd5 = getMd5(jobLink + jobTitle)

    if existJobList.count(jobMd5) > 0:
      GREEN ('Job exists in db')

    else: 
      YELLOW('[' + name + ']' + ' Adding......' + jobTitle)
      D['company'] = name
      D['job_link'] = jobLink
      D['job_title'] = jobTitle
      D['job_location'] = job.find ('span',class_='locationValue').string
      D['job_type'] = job.find('td',class_='sh_label').find_all('span')[0].string
      D['period_from'] = toDay()
      D['md5'] = jobMd5
      D['logo'] = self.logo
      try:
          jdPage = BeautifulSoup(getPage(jobLink,self.base_headers),features='html.parser')
          D['period_to'] = formatDate(jdPage.find('div', class_='Expertise').find_all('td')[0].contents[0].string),
          D['job_description'] = formatHTML(jdPage.find('div',class_='description'))
          addJob(D)
      except:
        D['error'] = traceback.format_exc()
        dumpError(D)
